In [1]:
import pandas as pd
import numpy as np
import requests
from bs4 import BeautifulSoup
import time
from splinter import Browser
import warnings
warnings.filterwarnings('ignore')
print('Libraries imported!')

Libraries imported!


#url = 'https://geocoder.ca/?locate='
#postalCode = 'T2L+0C9'/
#page = requests.get(url+postalCode)
#soup = BeautifulSoup(page.text, "html.parser")
#print(soup.prettify())

tables = pd.read_html(url+postalCode)
table_df = tables[1]
table_df.head()

title = soup.find('li', class_='pull-right').get_text()
print(title)

In [2]:
csvData = pd.read_csv('data/score_df.csv')
csvData.head()

,postal_code,walk_score,bike_score,transit_score
0,T2V 1M4,58.0,61.0,55.0
1,T2V 2G9,57.0,75.0,57.0
2,T2T 4L7,38.0,82.0,38.0
3,T2G 1M1,64.0,78.0,49.0
4,T3H 5Y7,65.0,89.0,40.0


In [6]:
csvDataPC = csvData['postal_code']
csvDataPC.head()
#testCSV = csvDataPC.head(20)

0    T2V 1M4
1    T2V 2G9
2    T2T 4L7
3    T2G 1M1
4    T3H 5Y7
Name: postal_code, dtype: object

In [12]:
len(csvDataPC)

2232

csvDataPC = csvDataPC.str.replace(' ', '+')
testCSV = csvDataPC.head(20)

coordinates = []
postalCodes = []
for i in csvDataPC:
    url = 'https://geocoder.ca/?locate='
    page = requests.get(url+str(i))
    time.sleep(5)
    soup = BeautifulSoup(page.text, "html.parser")
    try:
        title = soup.find('li', class_='pull-right').get_text()
    except:
        title = 'None'
    coordinates.append(title)
    #postalCodesP = i.str.replace('+', ' ')
    postalCodes.append(i)

coordinatesDF = pd.DataFrame(coordinates, postalCodes)
coordinatesDF.reset_index(inplace=True)

coordinatesDF.columns=['postal_codes', 'coordinates']
coordinatesDF.head()

In [7]:
import pgeocode

In [9]:
coordinates = []
postalCodes = []
for i in csvDataPC:
    country_code = pgeocode.Nominatim('ca')
    coord = country_code.query_postal_code(str(i))
    x = [coord.latitude, coord.longitude]
    coordinates.append(x)
    postalCodes.append(str(i))

In [10]:
coordinates

[[50.9819, -114.1004],
 [50.9819, -114.1004],
 [51.0242, -114.1004],
 [51.0272, -114.0349],
 [51.0419, -114.2],
 [51.1563, -114.0572],
 [51.1188, -113.9471],
 [50.9093, -114.1028],
 [50.9514, -114.3591],
 [51.1467, -114.3133],
 [51.0621, -114.1159],
 [50.987, -113.9634],
 [50.9899, -114.1574],
 [51.1467, -114.3133],
 [50.9278, -113.9682],
 [50.9278, -113.9682],
 [50.8796, -113.9555],
 [50.9278, -113.9682],
 [50.8796, -113.9555],
 [51.0233, -114.071],
 [51.1626, -113.9537],
 [50.8837, -114.0326],
 [50.8837, -114.0326],
 [50.9093, -114.1028],
 [51.0272, -114.0349],
 [51.0209, -113.981],
 [51.0621, -114.1159],
 [50.9899, -114.1574],
 [50.9278, -113.9682],
 [51.0708, -113.6931],
 [51.1563, -114.0572],
 [51.0419, -114.2],
 [51.1264, -114.1419],
 [50.9514, -114.3591],
 [51.133, -113.856],
 [51.1387, -114.2015],
 [51.0823, -113.9578],
 [50.8476, -114.1958],
 [51.2021, -114.2453],
 [51.1467, -114.3133],
 [51.2074, -114.1348],
 [51.133, -113.856],
 [51.1563, -114.0572],
 [51.0497, -114.1394],
 

In [11]:
len(coordinates)

2232

In [13]:
coordinatesDF = pd.DataFrame(coordinates, postalCodes)
coordinatesDF.reset_index(inplace=True)

In [16]:
coordinatesDF.head()

,postal_codes,lat,long
0,T2V 1M4,50.9819,-114.1004
1,T2V 2G9,50.9819,-114.1004
2,T2T 4L7,51.0242,-114.1004
3,T2G 1M1,51.0272,-114.0349
4,T3H 5Y7,51.0419,-114.2000


In [15]:
coordinatesDF.columns=['postal_codes', 'lat', 'long']
coordinatesDF.head()

,postal_codes,lat,long
0,T2V 1M4,50.9819,-114.1004
1,T2V 2G9,50.9819,-114.1004
2,T2T 4L7,51.0242,-114.1004
3,T2G 1M1,51.0272,-114.0349
4,T3H 5Y7,51.0419,-114.2000


In [17]:
coordinatesDF.to_csv('data/cal_coordinatesDF.csv', index=False)